In [16]:
import matplotlib.pyplot as plt
import nibabel
import numpy as np
import pandas as pd
from pathlib import Path
from skimage.transform import rescale
from tqdm import tqdm
import os
import researchpy
from glob import glob
import shutil

# LGG

#### In this notebook we compare the true volumes of the tumor label before and after skull-stripping.

In [43]:
list_sub = os.listdir('/mnt/public_data/preproc_study/gbm/1_reg_individual/')

In [63]:
gt_volumes = []
t1_volumes = []
t2_volumes = []
flair_volumes = []
ct1_volumes = []


def get_volume(zooms, mask):
    """Calculates true volume of binary mask """
    mask = mask.astype(bool)*1
    mask_sum = mask.sum().sum()
    volume = mask_sum * zooms[0] * zooms[1] * zooms[2]
    return volume


def get_volume_gbm(zooms, label):
    """Calculates true volume of BraTS mask"""
    y_wt, y_tc, y_et = label > 0, ((label == 1) + (label == 3)) > 0, label == 3
    mask_sum = np.sum(y_wt)
    volume = mask_sum * zooms[0] * zooms[1] * zooms[2]
    return volume/1000


def get_volume_gbm_masked(zooms, label, mask):
    """Calculates true volume of BraTS mask"""
    y_wt, y_tc, y_et = label > 0, ((label == 1) + (label == 3)) > 0, label == 3
    mask = mask[y_wt]
    mask_sum = np.sum(mask)
    volume = mask_sum * zooms[0] * zooms[1] * zooms[2]
    return volume/1000


for patient in tqdm(glob('/mnt/public_data/preproc_study/gbm/1_reg_individual/TCGA*')):
    """ For all directories in the GBM dataset calculating the tumor volumes after skull stripping.
    In each modality separately."""
    if Path(patient).is_dir():

        label = nibabel.load(str(patient) + '/CT1_SEG.nii.gz').get_fdata()
        label_zooms = nibabel.load(str(patient) + '/CT1_SEG.nii.gz').header.get_zooms()
        gt_volumes.append(get_volume_gbm(label_zooms, label))
        
        t1_mask = nibabel.load(str(patient) + '/T1_mask.nii.gz').get_fdata()
        t1_mask_zooms = nibabel.load(str(patient) + '/T1_mask.nii.gz').header.get_zooms()
        assert t1_mask_zooms == label_zooms
        t1_volumes.append(get_volume_gbm_masked(label_zooms, label, t1_mask))
        
        ct1_mask = nibabel.load(str(patient) + '/CT1_mask.nii.gz').get_fdata()
        ct1_mask_zooms = nibabel.load(str(patient) + '/CT1_mask.nii.gz').header.get_zooms()
        assert ct1_mask_zooms == label_zooms
        ct1_volumes.append(get_volume_gbm_masked(label_zooms, label, ct1_mask))
        
        flair_mask = nibabel.load(str(patient) + '/FLAIR_mask.nii.gz').get_fdata()
        flair_mask_zooms = nibabel.load(str(patient) + '/FLAIR_mask.nii.gz').header.get_zooms()
        assert flair_mask_zooms == label_zooms
        flair_volumes.append(get_volume_gbm_masked(label_zooms, label, flair_mask))
        
        t2_mask = nibabel.load(str(patient) + '/T2_mask.nii.gz').get_fdata()
        t2_mask_zooms = nibabel.load(str(patient) + '/T2_mask.nii.gz').header.get_zooms()
        assert t2_mask_zooms == label_zooms
        t2_volumes.append(get_volume_gbm_masked(label_zooms, label, t2_mask))

100%|██████████| 102/102 [1:06:45<00:00, 39.27s/it]


In [76]:
gt_errors = abs(np.array(gt_volumes) - np.array(ct1_volumes))/np.array(gt_volumes)
flair_errors = abs(np.array(gt_volumes) - np.array(flair_volumes))/np.array(gt_volumes)
t1_errors = abs(np.array(gt_volumes) - np.array(t1_volumes))/np.array(gt_volumes)
t2_errors = abs(np.array(gt_volumes) - np.array(t2_volumes))/np.array(gt_volumes)

In [77]:
np.mean(gt_errors), np.mean(flair_errors), np.mean(t1_errors), np.mean(t2_errors)

(0.0035365294817691086,
 0.002749536803530542,
 0.005887912752872096,
 0.002644775665931629)

In [78]:
from researchpy import ttest

def ttest_pair(df_1, df_2, name_1 = 'one', name_2 = 'two', correction = None):
    return ttest(pd.Series(df_1), pd.Series(df_2), 
#                  group1_name = name_1,
#                  group2_name= name_2, 
                 equal_variances=False, paired=True,)[1].iloc[4].values[1]

In [85]:
ttest_pair(gt_errors, t1_errors), ttest_pair(gt_errors, t2_errors), ttest_pair(gt_errors, flair_errors)

/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)
/home/kate/miniconda3/lib/python3.9/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(0.114, 0.0007, 0.0)

##### The cnclusion:

There is a statisticall difference on the errors with T1C and T1 and T2 image skull-stripping result, yet the difference is under 1%, so neglectable

# BGPD

In [86]:
list_sub = os.listdir('/mnt/public_data/preproc_study/bgpd/1_reg_individual/')

In [88]:
list(glob('/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/*'))

['/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/T1.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/T2_mask.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/CT1_mask.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/T1_mask.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/FLAIR_mask.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/CT1.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/mask_GTV_FLAIR.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/T2.nii.gz',
 '/mnt/public_data/preproc_study/bgpd/1_reg_individual/1043_18_4/FLAIR.nii.gz']

In [ ]:
gt_volumes = []
t1_volumes = []
t2_volumes = []
flair_volumes = []
ct1_volumes = []


def get_volume(zooms, mask):
    """Calculates true volume of binary mask """
    mask = mask.astype(bool)*1
    mask_sum = mask.sum().sum()
    volume = mask_sum * zooms[0] * zooms[1] * zooms[2]
    return volume/1000

def get_volume_masked(zooms, label, mask):
    """Calculates true volume of GTV mask"""
    mask = mask[label.astype(bool)]
    mask_sum = np.sum(mask)
    volume = mask_sum * zooms[0] * zooms[1] * zooms[2]
    return volume/1000


for patient in tqdm(glob('/mnt/public_data/preproc_study/bgpd/1_reg_individual/*')):
    """ For all directories in the BGPD dataset calculating the tumor volumes after skull stripping.
    In each modality separately."""
    if Path(patient).is_dir():

        label = nibabel.load(str(patient) + '/mask_GTV_FLAIR.nii.gz').get_fdata()
        label_zooms = nibabel.load(str(patient) + '/mask_GTV_FLAIR.nii.gz').header.get_zooms()
        gt_volumes.append(get_volume(label_zooms, label))
        
        t1_mask = nibabel.load(str(patient) + '/T1_mask.nii.gz').get_fdata()
        t1_mask_zooms = nibabel.load(str(patient) + '/T1_mask.nii.gz').header.get_zooms()
        assert t1_mask_zooms == label_zooms
        t1_volumes.append(get_volume_masked(label_zooms, label, t1_mask))
        
        ct1_mask = nibabel.load(str(patient) + '/CT1_mask.nii.gz').get_fdata()
        ct1_mask_zooms = nibabel.load(str(patient) + '/CT1_mask.nii.gz').header.get_zooms()
        assert ct1_mask_zooms == label_zooms
        ct1_volumes.append(get_volume_masked(label_zooms, label, ct1_mask))
        
        flair_mask = nibabel.load(str(patient) + '/FLAIR_mask.nii.gz').get_fdata()
        flair_mask_zooms = nibabel.load(str(patient) + '/FLAIR_mask.nii.gz').header.get_zooms()
        assert flair_mask_zooms == label_zooms
        flair_volumes.append(get_volume_masked(label_zooms, label, flair_mask))
        
        t2_mask = nibabel.load(str(patient) + '/T2_mask.nii.gz').get_fdata()
        t2_mask_zooms = nibabel.load(str(patient) + '/T2_mask.nii.gz').header.get_zooms()
        assert t2_mask_zooms == label_zooms
        t2_volumes.append(get_volume_masked(label_zooms, label, t2_mask))

 40%|████      | 72/180 [50:23<09:38,  5.36s/it]   